In [328]:
import spacy
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from collections import Counter

# nlp = spacy.load('en_core_web_sm')
nlpmd = spacy.load('en_core_web_md')
nlplg = spacy.load('en_core_web_lg')


In [329]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content":  msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [330]:
data.head()

,class_to_exec,content,date,from,subject,to
0,Booking,"Dear Rohit,\n \nAs per our conversation a whil...",None,"[(, Meemendra), (, ), (, meemendra.kumar@andbe...",Mr. Nagdev X 04,"[(, Gateway)]"
1,Booking,"Hi Agnes,\n \nI have not received a response t...",None,"[(, Kurien), (, Vinay), (, ), (, vkurien@jones...",RE: Rate Proposal - Taj Connemara,"[(Reservations, Reservations@tajhotels.com)]"
2,Booking,"As per corporate holiday plan, kindly arrange ...",None,"[(, tehp@tatasteel.com[mailto:tehp@tatasteel.c...",TEHP/18/12809: Booking request of Arun Gadamsh...,"[(Reservations Coorg, reservations.coorg@tajho..."
3,Booking,"Dear Gampa ji,\n \nGreetings of the day !\n \n...",None,"[(, INDIAN), (, ), (, indianjourney@gmail.com)]",Valley Period/3 double room/Lerner group,"[(, Neeraj)]"
4,Cancellation,Hi\n\nPlease cancel my booking .,None,"[(, rajivbhatia@ase.ae[mailto:rajivbhatia@ase....",Re: Taj Chandigarh Payment link for reservatio...,"[(Reservations, Reservations@tajhotels.com)]"


In [335]:
body = data["content"]

entityData = []
for cnt in body:
    entityCh = []
    contentSentence = nlplg(cnt)
    for sent in contentSentence.sents:
        sentence = str(sent).replace('\n', ' ')
        entity = nlplg(sentence)
        for token in entity.ents:
            entityData.append({"text":sentence, "Entity":token.text, "label":token.label_, "start_char":token.start_char, "end_char":token.end_char})
    #entityData.append({"body":cnt, "entities":entityCh})

In [336]:
entities = pd.DataFrame(entityData)

In [139]:
# print(entities["body"][0])

In [340]:
entities["Target"] = entities["Entity"].apply(lambda x: x.strip() != "")

In [343]:
entities = entities[entities["Target"]]

In [348]:
entities.to_csv("../datawe/raw/Email_Classification/entityTraining_17.csv")

In [174]:
# entities["Target"] = entities["Entity"].apply(lambda x: True if x.strip() != "" else False)

In [144]:
# entities.to_csv("../datawe/raw/Email_Classification/entityTraining.csv")

In [200]:
entityDf = pd.read_csv("../datawe/raw/Email_Classification/entityTraining.csv")

In [195]:
entityDf

,Entity,end_char,label,start_char,text,Target
0,Rohit,10,PERSON,5,"Dear Rohit,\n \nAs per our conversation a whil...",True
1,02 Executive room,92,PRODUCT,75,"Dear Rohit,\n \nAs per our conversation a whil...",True
2,Taj Gateway Ganges,114,ORG,96,"Dear Rohit,\n \nAs per our conversation a whil...",True
3,24/26 December 2018,139,DATE,120,"Dear Rohit,\n \nAs per our conversation a whil...",True
4,two nights,154,DATE,144,"Dear Rohit,\n \nAs per our conversation a whil...",True
5,Agnes,8,PERSON,3,"Hi Agnes,\n \nI have not received a response t...",True
6,Hotel/Resort,102,FAC,90,"As per corporate holiday plan, kindly arrange ...",True
7,TEHP/18/12809,161,CARDINAL,147,"As per corporate holiday plan, kindly arrange ...",True
8,\nArun Gadamshetty,184,PERSON,167,"As per corporate holiday plan, kindly arrange ...",True
9,\nSpares Manufacturing Department,244,ORG,212,"As per corporate holiday plan, kindly arrange ...",True


In [17]:
entityDf['Entity'][14]

'Vivanta'

## creating model

In [218]:
# training data
TRAIN_DATA = [
    ('Who is Shaka Khan?', {
        'entities': [(7, 17, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]
# entityTrainData

# TRAIN_DATA = entityTrainData

In [219]:
entityTrainData

[('Contact No\n:\n8092084556\nPlace\n:\nVivanta by Taj - Madikeri Coorg, Coorg\nRoom Type\n:\nSuperior Charm\nCheck',
  {'entities': [(39, 31, 'PERSON'),
    (57, 43, 'ORG'),
    (63, 58, 'GPE'),
    (80, 65, 'ORG')]}),
 ('Dear Rohit,\n \nAs per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.',
  {'entities': [(10, 5, 'PERSON'),
    (91, 75, 'PRODUCT'),
    (114, 96, 'ORG'),
    (139, 120, 'DATE'),
    (154, 144, 'DATE')]}),
 ('No of rooms\n:\n1\nComments by requester\n:\nFor Honeymoon trip\n\n\nTraveller Details:\n1. \n',
  {'entities': [(12, 11, 'CARDINAL'), (58, 44, 'WORK_OF_ART')]}),
 ('SHWETA GADAMSHETTY\n\n', {'entities': [(18, 0, 'GPE')]}),
 ('Name\n:\nArun Gadamshetty\n\n', {'entities': [(23, 7, 'PERSON')]}),
 ('In Date-Time\n:\n10 Dec 2018 02:00 PM\nCheck Out',
  {'entities': [(26, 15, 'DATE'), (36, 27, 'TIME')]}),
 ('Hi Agnes,\n \nI have not received a response to this email as yet.',
  {'e

In [303]:
nlp = spacy.blank('en')

In [304]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

In [307]:
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
#         for ents in ent:
#             ner.add_label(ents[2])
# #             print(ents)
# #         print(ent)
        ner.add_label(ent[2])

TRAIN_DATA

[('Dear Rohit,   As per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.',
  {'entities': [('PERSON',
     'Dear Rohit,   As per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.',
     5),
    ('PRODUCT',
     'Dear Rohit,   As per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.',
     75),
    ('ORG',
     'Dear Rohit,   As per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.',
     96),
    ('DATE',
     'Dear Rohit,   As per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.',
     120),
    ('DATE',
     'Dear Rohit,   As per our conversation a while ago, pleas

In [306]:
model=None; output_dir=None; n_iter=100;
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print('Losses', losses)

TypeError: 'str' object cannot be interpreted as an integer

In [225]:
# test the trained model
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('London', 'LOC'), ('Berlin', 'LOC')]
Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3), ('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]
Entities [('Shaka Khan', 'PERSON')]
Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3), ('Khan', 'PERSON', 1), ('?', '', 2)]


In [217]:
# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

    # test the saved model
    print("Loading from", output_dir)
    nlp2 = spacy.load(output_dir)
    for text, _ in TRAIN_DATA:
        doc = nlp2(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [22]:
# #!/usr/bin/env python
# # coding: utf8
# """Example of training spaCy's named entity recognizer, starting off with an
# existing model or a blank model.

# For more details, see the documentation:
# * Training: https://spacy.io/usage/training
# * NER: https://spacy.io/usage/linguistic-features#named-entities

# Compatible with: spaCy v2.0.0+
# """
# from __future__ import unicode_literals, print_function

# import plac
# import random
# from pathlib import Path
# import spacy
# from spacy.util import minibatch, compounding

In [28]:



# # training data
# TRAIN_DATA = [
#     ('Who is Shaka Khan?', {
#         'entities': [(7, 17, 'PERSON')]
#     }),
#     ('I like London and Berlin.', {
#         'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
#     })
# ]


# @plac.annotations(
#     model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#     output_dir=("Optional output directory", "option", "o", Path),
#     n_iter=("Number of training iterations", "option", "n", int))
# def mainT(model=None, output_dir=None, n_iter=100):
#     """Load the model, set up the pipeline and train the entity recognizer."""
#     if model is not None:
#         nlp = spacy.load(model)  # load existing spaCy model
#         print("Loaded model '%s'" % model)
#     else:
#         nlp = spacy.blank('en')  # create blank Language class
#         print("Created blank 'en' model")

#     # create the built-in pipeline components and add them to the pipeline
#     # nlp.create_pipe works for built-ins that are registered with spaCy
#     if 'ner' not in nlp.pipe_names:
#         ner = nlp.create_pipe('ner')
#         nlp.add_pipe(ner, last=True)
#     # otherwise, get it so we can add labels
#     else:
#         ner = nlp.get_pipe('ner')

#     # add labels
#     for _, annotations in TRAIN_DATA:
#         for ent in annotations.get('entities'):
#             ner.add_label(ent[2])

#     # get names of other pipes to disable them during training
#     other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
#     with nlp.disable_pipes(*other_pipes):  # only train NER
#         optimizer = nlp.begin_training()
#         for itn in range(n_iter):
#             random.shuffle(TRAIN_DATA)
#             losses = {}
#             # batch up the examples using spaCy's minibatch
#             batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
#             for batch in batches:
#                 texts, annotations = zip(*batch)
#                 nlp.update(
#                     texts,  # batch of texts
#                     annotations,  # batch of annotations
#                     drop=0.5,  # dropout - make it harder to memorise data
#                     sgd=optimizer,  # callable to update weights
#                     losses=losses)
#             print('Losses', losses)

#     # test the trained model
#     for text, _ in TRAIN_DATA:
#         doc = nlp(text)
#         print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#         print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

#     # save model to output directory
#     if output_dir is not None:
#         output_dir = Path(output_dir)
#         if not output_dir.exists():
#             output_dir.mkdir()
#         nlp.to_disk(output_dir)
#         print("Saved model to", output_dir)

#         # test the saved model
#         print("Loading from", output_dir)
#         nlp2 = spacy.load(output_dir)
#         for text, _ in TRAIN_DATA:
#             doc = nlp2(text)
#             print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#             print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])



In [41]:

# # if __name__ == '__main__':
# plac.call(mainT)

#     # Expected output:
#     # Entities [('Shaka Khan', 'PERSON')]
#     # Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3),
#     # ('Khan', 'PERSON', 1), ('?', '', 2)]
#     # Entities [('London', 'LOC'), ('Berlin', 'LOC')]
#     # Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3),
#     # ('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]

In [295]:
# training data
TRAIN_DATA = [
    ('Who is Shaka Khan?', {
        'entities': [(7, 17, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]



In [294]:
# entityDf["text"][0]

In [293]:
# entityDf.columns

In [111]:
# entityTrainData = []
# entityDfTrain = entityDfTrain[entityDfTrain["Target"]]
# for entityTrain in  entityDfTrain.values:
#     entityTrainData.append((entityTrain[5], {'entities':[(entityTrain[2], entityTrain[4], entityTrain[3])]}))

In [184]:
# entityTrainData

In [183]:
# entityDf['Entity'] = entityDf['Entity'].apply(lambda x: )

In [283]:
entityDfTrain = entityDf[entityDf.apply(lambda x: x["text"][x["start_char"]:x["end_char"]] == x["Entity"], axis=1)]
unique = entityDfTrain['text'].value_counts()

In [292]:
# type(unique)

In [296]:
entityTrainData = []
for text, item in  unique.items():
#     print(item, text)
    mulItems = entityDfTrain[entityDfTrain['text'] == text]
    multipleEntities = []
    for dta in mulItems.values:
        multipleEntities.append((dta[2], dta[4], dta[3]))
#          entityTrainData.append((dta[5], ))
    entityTrainData.append((text, {'entities':multipleEntities}))
    
TRAIN_DATA = entityTrainData

In [297]:
# len(entityTrainData)

In [298]:
# entityDf.columns

In [299]:
# for item in entityDf.values:
#     if
#     print(item[4][item[3]:item[1]] == item[0], ' ==> ' , item[0], '   ', item[4][item[3]:item[1]], len(item[0]), len(item[4][item[3]:item[1]]))
# #     print(item[2])

In [300]:
# entityDf[entityDf["Target"]].to_csv("../datawe/raw/Email_Classification/entityTraining.csv")

In [301]:
#  entityDf.drop([entityDf.columns[0]], inplace=True)

In [302]:
# entityDf.columns

In [179]:
len('SHWETA GADAMSHETTY')

18

In [262]:
txt = """"Contact No : 8092084556 Place : Vivanta by Taj - Madikeri Coorg, Coorg Room Type : Superior Charm Check"""
idx = "Taj - Madikeri Coorg"
txt.index(idx), txt.index(idx) + len(idx)

(44, 64)

In [241]:
entityTrainData[0][0].replace('\n' , ' ')

'Contact No : 8092084556 Place : Vivanta by Taj - Madikeri Coorg, Coorg Room Type : Superior Charm Check'

In [249]:
for dta in entityDf.values:
#     print(dta[4])
    dta[4] = dta[4].replace('\n', ' ')

In [254]:
entityDf['text'] = entityDf['text'].apply(lambda x: x.replace('\n', ' '))

In [255]:
entityDf['text']

0     Dear Rohit,   As per our conversation a while ...
1     Dear Rohit,   As per our conversation a while ...
2     Dear Rohit,   As per our conversation a while ...
3     Dear Rohit,   As per our conversation a while ...
4     Dear Rohit,   As per our conversation a while ...
5     Hi Agnes,   I have not received a response to ...
6     As per corporate holiday plan, kindly arrange ...
7     As per corporate holiday plan, kindly arrange ...
8     As per corporate holiday plan, kindly arrange ...
9     As per corporate holiday plan, kindly arrange ...
10    Contact No : 8092084556 Place : Vivanta by Taj...
11    Contact No : 8092084556 Place : Vivanta by Taj...
12    Contact No : 8092084556 Place : Vivanta by Taj...
13    Contact No : 8092084556 Place : Vivanta by Taj...
14          In Date-Time : 10-Dec-18 02:00 PM Check Out
15        In Date-Time : 10 Dec 2018 02:00 PM Check Out
16                      Date-Time : 12-Dec-18 12:00 PM 
17                    Date-Time : 12 Dec 2018 12

In [270]:
entityDf[entityDf.apply(lambda x: x["text"][x["start_char"]:x["end_char"]] == x["Entity"], axis=1)]

,Entity,end_char,label,start_char,text,Target
0,Rohit,10,PERSON,5,"Dear Rohit, As per our conversation a while ...",True
1,02 Executive room,92,PRODUCT,75,"Dear Rohit, As per our conversation a while ...",True
2,Taj Gateway Ganges,114,ORG,96,"Dear Rohit, As per our conversation a while ...",True
3,24/26 December 2018,139,DATE,120,"Dear Rohit, As per our conversation a while ...",True
4,two nights,154,DATE,144,"Dear Rohit, As per our conversation a while ...",True
5,Agnes,8,PERSON,3,"Hi Agnes, I have not received a response to ...",True
6,Hotel/Resort,102,FAC,90,"As per corporate holiday plan, kindly arrange ...",True
11,Taj - Madikeri,57,ORG,43,Contact No : 8092084556 Place : Vivanta by Taj...,True
12,Coorg,63,GPE,58,Contact No : 8092084556 Place : Vivanta by Taj...,True
14,10-Dec-18,24,DATE,15,In Date-Time : 10-Dec-18 02:00 PM Check Out,True


In [323]:
# don't forget to install a model, e.g.: python -m spacy download en
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_lg")

LONG_NEWS_ARTICLE = "I love everything you put out. This and every other thing NLP related, is helping me start a small venture that I hope to one day turn into a small business. Do you have any other published work or papers that you could share?."

doc = nlp(u''+ LONG_NEWS_ARTICLE)
displacy.render(doc, style='dep', jupyter=True)

# doc2 = nlp(LONG_NEWS_ARTICLE)
# displacy.render(doc2, style='ent', jupyter=True)

In [324]:
doc2 = nlp( LONG_NEWS_ARTICLE)
displacy.render(doc2, style='ent', jupyter=True)